In [ ]:
from itertools import compress
import pandas as pd
import re

In [ ]:
#importando planilha do portal admin para tratamento da base

df_office=pd.read_csv("office2.csv", low_memory=False)
df_office.head()
df_office.columns


In [ ]:
#Seleção da colunas necessarias apenas

cols=[ 'Departamento','Nome','Sobrenome', 'Licenças',
      'Nome UPN']

df_office=df_office[cols]
df_office.head()

df_office=df_office.query("@pd.notna(Licenças)").reset_index(drop=True)
df_office.head()


In [ ]:
df_office.head()
df = df_office.copy()

In [ ]:
# licença email
email_license = [
    'Office 365 E1',
    'Office 365 E3',
    'Exchange Online (Plan 1)',
    'Microsoft 365 E3'
]

# power bi
powerbi_license = [
    'Power BI Pro',
    'Power BI (free)',
    'Power BI PRO'
]

# Project plan
project_plan_license = [
    'Project Plan 1',
    'Project Plan 5',
    'Project Plan 3'
]

# visio plan
visio_plan = [
    'Visio Plan 2'
]


In [ ]:
df['Nome UPN'][0].split('@')[0]

In [ ]:
df = df.assign(
    #Extraindo logins
    login = df['Nome UPN'].apply( lambda x: x.split('@')[0] ),

    # Criando as Colunas de licenca 
    email = df['Licenças'].apply( lambda x: str( list( compress( email_license, [ i in x for i in email_license ] ) ) ).replace('[', '').replace(']', '').replace("'", "") ),
    powerbi = df['Licenças'].apply( lambda x: str( list( compress( powerbi_license, [ i in x for i in powerbi_license ] ) ) ).replace('[', '').replace(']', '').replace("'", "") ),
    project_plan = df['Licenças'].apply( lambda x: str( list( compress( project_plan_license, [ i in x for i in project_plan_license ] ) ) ).replace('[', '').replace(']', '').replace("'", "") ),
    visio_plan = df['Licenças'].apply( lambda x: str( list( compress( visio_plan, [ i in x for i in visio_plan ] ) ) ).replace('[', '').replace(']', '').replace("'", "") )
)


In [ ]:
df_user = pd.read_excel('cc.xlsx')

df_user.head()

df.head()

In [ ]:
#Procv com planilha do DP para inclusão do CC
df = df.merge( df_cc, how='left', right_on='login', left_on='login')

In [ ]:
df_valores = pd.read_excel('valores.xlsx')
df.head()

In [ ]:
#Criando coluna com valores de cada licença
df_final = df.merge(df_valores, how='left', left_on='email', right_on='Licença e-mail') \
    .rename(columns={'Valor': 'vl_licenca_email'}) \
    .merge(df_valores, how='left', left_on='powerbi', right_on='Licença e-mail') \
    .rename(columns={'Valor': 'vl_licenca_power_bi'}) \
    .merge(df_valores, how='left', left_on='project_plan', right_on='Licença e-mail') \
    .rename(columns={'Valor': 'vl_licenca_projec_online'}) \
    .merge(df_valores, how='left', left_on='visio_plan', right_on='Licença e-mail') \
    .rename(columns={'Valor': 'vl_licenca_vision_online'}) \
    .drop('Licença e-mail_y', axis=1).rename(columns={'Licença e-mail_x': 'Licença e-mail'})

In [ ]:
#IF Else, percoreendo as linhas e separando as licenças em cada coluna 
df_final['vl_licenca_email'] = df_final['vl_licenca_email'].apply(lambda x: 0 if pd.isna(x) else x)
df_final['vl_licenca_power_bi'] = df_final['vl_licenca_power_bi'].apply(lambda x: 0 if pd.isna(x) else x)
df_final['vl_licenca_projec_online'] = df_final['vl_licenca_projec_online'].apply(lambda x: 0 if pd.isna(x) else x)
df_final['vl_licenca_vision_online'] = df_final['vl_licenca_vision_online'].apply(lambda x: 0 if pd.isna(x) else x)

df_final['vl_total'] = df_final['vl_licenca_email'] + df_final['vl_licenca_power_bi'] + df_final['vl_licenca_projec_online'] + df_final['vl_licenca_vision_online']

In [ ]:
df_final.columns

#organizando colunas  e selecionando as necessarias 
cols = [ 
     'login'
    ,'Funcionario'
    ,'Nome UPN'
    ,'Código do CDC'
    ,'Centro de Custo'
    ,'Cargo'
    ,'Licenças'
    ,'email'
    ,'powerbi'
    ,'project_plan'
    ,'visio_plan'
    ,'vl_licenca_email'
    ,'vl_licenca_power_bi'
    ,'vl_licenca_projec_online'
    ,'vl_licenca_vision_online'
    ,'vl_total'
]

df_final = df_final[cols]




In [ ]:
df_final.to_excel('final.xlsx')